# NGFC Modelling with IDEAS - pse
This is a modelling example of a NGFC system using idaes-pse framework.

In [1]:
from pyomo.environ import ConcreteModel, Constraint, Objective, SolverFactory, TransformationFactory, Constraint, Var
from pyomo.network import Arc

In [2]:
from idaes.core import FlowsheetBlock
from idaes.unit_models import Mixer, HeatExchanger, Separator, GibbsReactor
# Methane combustion ideal package got CH4, H2O, CO, CO2, N2, NH3, O2
import idaes.property_models.activity_coeff_models.methane_combustion_ideal as thermo_props
#import idaes.property_models.activity_coeff_models.methane_combustion_ideal as reaction_props

In [3]:
import matplotlib.pyplot as plt

### Building Base Flowsheet.

In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.thermo_params = thermo_props.MethaneParameterBlock()

In [5]:
# Fuel ultilization (Uf): mole reductant consumed in FC per mole of reductant total
Uf = 0.8
# Air ultilization (Ua): mole of air consumed in FC per mole of air feed
Ua = 0.2
# Methane to steam ratio (MS): mole methane per mole water
MS = 2
# Feed:
# Reaction: 
# Reforming: CH4 + H2O -> CO + 3H2
# Water gas shift: CO + H2O -> CO2 + H2
# Methane combustion: CH4 + 2O2 -> CO2 + 2H2O
# Hydrogen combustion: H2 + 1/2O2 -> H2O
# Carbon monoxide combustion: CO + 1/2O2 -> CO2

n_CH4f = 10
print("mole of methane feed: "+str(n_CH4f)+" mole/s")
n_H2Of = n_CH4f*MS
print("mole of steam feed: "+str(n_H2Of)+" mole/s")
n_O2f = n_CH4f*Uf*2/Ua
n_N2f = n_O2f*0.79/0.21
print("mole of air feed: "+str(n_N2f+n_O2f)+" mole/s")

n_H2ex = 2
n_COex = n_CH4f*(1-Uf)*4-n_H2ex
n_CO2ex = n_CH4f-n_COex
n_H2Oex = n_H2Of+2*n_CH4f-n_H2ex
y_H2ex = n_H2ex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_COex = n_COex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_CO2ex = n_CO2ex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_H2Oex = n_H2Oex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)

print("Anode exhaust: ")
print("y_H2ex: "+str(y_H2ex))
print("y_COex: "+str(y_COex))
print("y_CO2ex: "+str(y_CO2ex))
print("y_H2Oex: "+str(y_H2Oex))
print("Total mole/s: "+str(n_H2ex + n_COex + n_CO2ex + n_H2Oex))

n_N2ex = n_N2f
n_O2ex = n_O2f - n_CH4f*Uf*2
y_O2ex = n_O2ex/(n_O2ex+n_N2ex)
y_N2ex = n_N2ex/(n_O2ex+n_N2ex)
print("Cathode exhaust: ")
print("y_O2ex: "+str(y_O2ex))
print("y_N2ex: "+str(y_N2ex))
print("Total mole/s: "+str(n_O2ex+n_N2ex))

mole of methane feed: 10 mole/s
mole of steam feed: 20 mole/s
mole of air feed: 380.95238095238096 mole/s
Anode exhaust: 
y_H2ex: 0.04
y_COex: 0.11999999999999997
y_CO2ex: 0.08000000000000003
y_H2Oex: 0.76
Total mole/s: 50.0
Cathode exhaust: 
y_O2ex: 0.1753653444676409
y_N2ex: 0.824634655532359
Total mole/s: 364.95238095238096


### Mixing anode & cathode exhaust:

In [6]:
m.fs.mix_exhaust = Mixer(default={"dynamic": False,
                                  "property_package": m.fs.thermo_params})

In [7]:
# First inlet to mix_exhaust : anode exhaust
m.fs.mix_exhaust.inlet_1.flow_mol.fix(50.0)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"H2"].fix(0.04)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"CO"].fix(0.12)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"CO2"].fix(0.08)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"H2O"].fix(0.76)
m.fs.mix_exhaust.inlet_1.temperature.fix(700+273.15)
m.fs.mix_exhaust.inlet_1.pressure.fix(101325)

In [8]:
m.fs.mix_exhaust.inlet_1.display()

inlet_1 : Size=1
    Key  : Name           : Value
    None :       flow_mol : {0.0: 50.0}
         : mole_frac_comp : {(0.0, 'CH4'): 0.0, (0.0, 'CO'): 0.12, (0.0, 'CO2'): 0.08, (0.0, 'H2'): 0.04, (0.0, 'H2O'): 0.76, (0.0, 'N2'): 0.0, (0.0, 'NH3'): 0.0, (0.0, 'O2'): 0.0}
         :       pressure : {0.0: 101325}
         :    temperature : {0.0: 973.15}


In [9]:
# Second inlet to mix_exhaust : cathode exhaust
m.fs.mix_exhaust.inlet_2.flow_mol.fix(364.95238095238096)
m.fs.mix_exhaust.inlet_2.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix_exhaust.inlet_2.mole_frac_comp[0.0,"O2"].fix(0.1753653444676409)
m.fs.mix_exhaust.inlet_2.mole_frac_comp[0.0,"N2"].fix(0.824634655532359)
m.fs.mix_exhaust.inlet_2.temperature.fix(700+273.15)
m.fs.mix_exhaust.inlet_2.pressure.fix(101325)

In [10]:
m.fs.mix_exhaust.inlet_2.display()

inlet_2 : Size=1
    Key  : Name           : Value
    None :       flow_mol : {0.0: 364.95238095238096}
         : mole_frac_comp : {(0.0, 'CH4'): 0.0, (0.0, 'CO'): 0.0, (0.0, 'CO2'): 0.0, (0.0, 'H2'): 0.0, (0.0, 'H2O'): 0.0, (0.0, 'N2'): 0.824634655532359, (0.0, 'NH3'): 0.0, (0.0, 'O2'): 0.1753653444676409}
         :       pressure : {0.0: 101325}
         :    temperature : {0.0: 973.15}


### Burner as Gibbs Reactor

In [11]:
m.fs.burner = GibbsReactor(default={"dynamic": False,
                                     "property_package": m.fs.thermo_params,
                                     "has_pressure_change": False,
                                     "has_heat_transfer": True})

### Point mixer stream to Burner:

In [12]:
m.fs.stream1 = Arc(source=m.fs.mix_exhaust.outlet,
                  destination=m.fs.burner.inlet)

In [13]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [16]:
m.fs.burner.inlet.temperature.fix(700+273.15)

In [17]:
from idaes.core.util.model_statistics import degrees_of_freedom as dof
dof(m)

0

In [ ]:
m.fs.burner.inlet.flow_mol.fix(1000.0)
m.fs.burner.inlet.mole_frac_comp[0.0,:].fix(0.0)
m.fs.burner.inlet.mole_frac_comp[0.0,"H2"].fix(0.04)
m.fs.burner.inlet.mole_frac_comp[0.0,"CO"].fix(0.12)
m.fs.burner.inlet.mole_frac_comp[0.0,"CO2"].fix(0.08)
m.fs.burner.inlet.mole_frac_comp[0.0,"H2O"].fix(0.76)
m.fs.burner.inlet.temperature.fix(700+273.15)
m.fs.burner.inlet.pressure.fix(101325)

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom as dof
dof(m.fs.burner)

In [18]:
solver = SolverFactory('ipopt')

In [19]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      499
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      137

Total number of variables............................:      145
                     variables with only lower bounds:        8
                variables with lower and upper bounds:       64
                     variables with only upper bounds:        0
Total

 175  0.0000000e+00 4.23e-04 2.72e+06  -3.8 5.00e+04    -  1.00e+00 3.91e-03h  9
 176  0.0000000e+00 4.23e-04 2.72e+06  -3.8 5.00e+04    -  1.00e+00 3.91e-03h  9
 177  0.0000000e+00 4.23e-04 2.72e+06  -3.8 5.01e+04    -  1.00e+00 3.91e-03h  9
 178  0.0000000e+00 9.15e-02 1.63e+02  -3.8 5.01e+04    -  1.00e+00 1.00e+00w  1
 179  0.0000000e+00 1.44e-01 1.68e+06  -3.8 6.30e+04    -  3.85e-01 1.00e+00w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180  0.0000000e+00 2.33e-01 2.08e+06  -3.8 8.00e+04    -  5.27e-01 1.00e+00w  1
 181  0.0000000e+00 4.22e-04 2.72e+06  -3.8 1.05e+05    -  1.00e+00 3.91e-03h  8
 182  0.0000000e+00 4.22e-04 2.72e+06  -3.8 5.01e+04    -  1.00e+00 3.91e-03h  9
 183  0.0000000e+00 4.22e-04 2.72e+06  -3.8 5.01e+04    -  1.00e+00 3.91e-03h  9
 184  0.0000000e+00 4.22e-04 2.72e+06  -3.8 5.02e+04    -  1.00e+00 3.91e-03h  9
 185  0.0000000e+00 4.21e-04 2.72e+06  -3.8 5.02e+04    -  1.00e+00 3.91e-03h  9
 186  0.0000000e+00 4.21e-04

 316r 0.0000000e+00 3.53e-06 1.21e+04  -5.7 2.22e-02    -  1.11e-04 8.08e-03f  1
 317r 0.0000000e+00 3.34e-06 1.21e+04  -5.7 5.93e-02    -  2.38e-03 4.37e-03f  1
 318r 0.0000000e+00 2.21e-06 1.20e+04  -5.7 1.51e-01    -  6.21e-04 2.41e-03f  1
 319r 0.0000000e+00 4.96e-07 1.20e+04  -5.7 2.08e-01    -  2.58e-03 3.36e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 320r 0.0000000e+00 4.95e-07 1.20e+04  -5.7 1.17e-02    -  1.57e-02 1.93e-03f  1
 321r 0.0000000e+00 4.70e-07 1.18e+04  -5.7 6.46e-02    -  2.27e-04 1.26e-02f  1
 322r 0.0000000e+00 4.12e-07 1.18e+04  -5.7 2.41e-02    -  3.22e-03 5.97e-03f  1
 323r 0.0000000e+00 1.10e-08 1.17e+04  -5.7 2.38e-01    -  2.15e-03 4.01e-03f  1
 324r 0.0000000e+00 9.73e-09 1.17e+04  -5.7 1.33e-02    -  2.85e-01 5.40e-03f  2

Number of Iterations....: 324

                                   (scaled)                 (unscaled)
Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infea

In [20]:
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 145
  Number of variables: 145
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Found feasible point for square problem.
  Termination condition: optimal
  Id: 2
  Error rc: 0
  Time: 0.4812049865722656
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [21]:
m.fs.burner.inlet.display()

inlet : Size=1
    Key  : Name           : Value
    None :       flow_mol : {0.0: 414.95238095293416}
         : mole_frac_comp : {(0.0, 'CH4'): 0.0, (0.0, 'CO'): 0.01445949049331082, (0.0, 'CO2'): 0.009639660316759722, (0.0, 'H2'): 0.004819830166632905, (0.0, 'H2O'): 0.09157677301565598, (0.0, 'N2'): 0.7252696809805834, (0.0, 'NH3'): 0.0, (0.0, 'O2'): 0.15423456507617558}
         :       pressure : {0.0: 101324.9995}
         :    temperature : {0.0: 973.15}


In [22]:
m.fs.burner.outlet.display()

outlet : Size=1
    Key  : Name           : Value
    None :       flow_mol : {0.0: 416.2350027969305}
         : mole_frac_comp : {(0.0, 'CH4'): 4.251358500527426e-16, (0.0, 'CO'): 0.0033530164780888297, (0.0, 'CO2'): 0.020671863480689424, (0.0, 'H2'): 0.022029912159103864, (0.0, 'H2O'): 0.07406960129796497, (0.0, 'N2'): 0.7230347649033041, (0.0, 'NH3'): 7.408960167455998e-09, (0.0, 'O2'): 0.15684083427188822}
         :       pressure : {0.0: 101324.9995}
         :    temperature : {0.0: 3620.4362434597333}


In [ ]:
m.fs.mix = Mixer(default={"dynamic": False,
                          "property_package": m.fs.thermo_params})

In [ ]:
m.fs.seperator = Separator(default={"dynamic": False,
                                    "property_package": m.fs.thermo_params})